# Language Models evaluated on the data of Talafeef

## Trained, tested, evaluated models using classification metrics
  - CRF Model: (crf_model.sav)
  - LSTM (RNN) Model: (RNN-model.h5, RNN_tag2index.pkl, RNN_word2index)

## Word embeddings using Skip-Gram, CBOW, and araBERTv02:
  - Skip-Gram Model: (SkipGram_model.pt)
  - CBOW Model: (cbow_model.h5 and CBOW_Embeddings.npz)
  - araBERTv02 (source: https://huggingface.co/aubmindlab/bert-base-arabertv2): (token_vecs_cat_array.pkl, token_vecs_cat_array.npz, tokenized_text.pkl)

## Fine-tuned pre-trained araBERTv02 Model on the Project-specific Data
  - fine_tuned_arabertv02.zip

In [ ]:
!pip install lazyme

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for lazyme: filename=lazyme-0.0.27-py3-none-any.whl size=8606 sha256=8eb3a44cec218cb1c5d076920621b46d6003ed030abe2778949f3aabeb458ad8
  Stored in directory: /root/.cache/pip/wheels/69/d3/f1/ba84128432d5b0c2408d2ae86ea09d743e28e4d50aca31d39e
Successfully built lazyme


# 1. 1. Random Fields (CRF)

In [ ]:
from lazyme import per_section
import nltk 
from collections import Counter
import numpy as np
import pandas as pd

In [ ]:
dataset = pd.read_csv("/content/Talafeef.csv",usecols=['token','tpos'])
dataset.head()
dataset.isnull().sum()

token    0
tpos     0
dtype: int64

In [ ]:
dataset.isnull().sum()

token    0
tpos     0
dtype: int64

In [ ]:
dataset['tpos'].apply(lambda x:type(x)).value_counts()

<class 'str'>    61317
Name: tpos, dtype: int64

In [ ]:
arabic_period_count = dataset.query('token=="."').shape[0]
print(f"Number of Periods: {arabic_period_count}")

Number of Periods: 1412


In [ ]:
#end of the sentences
end_sent_index = dataset.query('token=="."').index

In [ ]:
# Splitting the data into sentences using the period (.) as the unique identifier
sentences = []
sentence = []
for index, row in dataset.iterrows():
    sentence.append((row['token'], row['tpos']))  # Add both token and tpos as a tuple
    if row['token'] == '.':
        sentences.append(sentence)
        sentence = []

In [ ]:
arabic_comma_sentences = [s for s in sentences if any(t[0] == '،' for t in s)]
print(arabic_comma_sentences[:10])

[[('مثال', 'N'), ('الصورة', 'N'), ('34', 'CD'), (':', 'PUNC'), ('صورتان', 'N'), ('شعاعيتان', 'N'), ('متطابقتان', 'JJ'), ('ل', 'IN'), ('القسم', 'N'), ('العلوي', 'JJ'), ('من', 'IN'), ('البطن', 'N'), ('تظهران', 'VBP'), ('غاز', 'N'), ('في', 'IN'), ('الشجرة', 'N'), ('الصفراوية', 'JJ'), ('،', 'PUNC'), ('يوجد', 'VBP'), ('تفرع', 'N'), ('خطوط', 'N'), ('سوداء', 'JJ'), ('غاز', 'N'), ('بارز', 'JJ'), ('نحو', 'N'), ('مركز', 'N'), ('الكبد', 'N'), ('أكبر', 'JJR'), ('و', 'CC'), ('أشد', 'JJR'), ('بروزا', 'N'), ('نحو', 'N'), ('السرة', 'N'), ('،', 'PUNC'), ('و', 'CC'), ('يوجد', 'VBP'), ('أيضا', 'N'), ('شبكة', 'N'), ('صفراوية', 'JJ'), ('واقعة', 'JJ'), ('على', 'IN'), ('الخط', 'N'), ('الناصف', 'JJ'), ('الأسهم', 'N'), ('البيضاء', 'N'), ('و', 'CC'), ('التي', 'WP'), ('تتوضع', 'VBP'), ('في', 'IN'), ('داخل', 'JJ'), ('القناة', 'N'), ('الصفراوية', 'JJ'), ('المشتركة', 'JJ'), ('و', 'CC'), ('هذا', 'DT'), ('يفسر', 'VBP'), ('أين', 'WRB'), ('يستطيع', 'VBP'), ('الغاز', 'N'), ('أن', 'RP'), ('ينتقل', 'VBP'), ('ب', 'IN'), ('

In [ ]:
print(len(sentences))

1412


In [ ]:
dataset_size = np.arange(len(sentences))
train_index = np.random.choice(dataset_size, int(len(sentences) * 0.8), replace=False)
test_index = np.setdiff1d(dataset_size, train_index)

In [ ]:
train_index.shape

(1129,)

In [ ]:
test_index.shape

(283,)

In [ ]:
training_sentences = [sentences[i] for i in train_index]
test_sentences = [sentences[i] for i in test_index]

In [ ]:
print (training_sentences[0])

[('اللهم', 'N'), ('من', 'RP'), ('أراد', 'VBD'), ('بلادنا', 'N'), ('و', 'CC'), ('ديننا', 'N'), ('و', 'CC'), ('عقيدتنا', 'N'), ('و', 'CC'), ('أمننا', 'N'), ('ب', 'IN'), ('سوء', 'N'), ('و', 'CC'), ('كيد', 'N'), ('ف', 'CC'), ('اردد', 'VBP'), ('كيده', 'N'), ('في', 'IN'), ('نحره', 'N'), ('و', 'CC'), ('أشغله', 'VB'), ('ب', 'IN'), ('نفسه', 'N'), ('يا', 'RP'), ('رب', 'N'), ('العالمين', 'N'), ('.', 'PUNC')]


In [ ]:
def features(sentence, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    return {
        'word': sentence[index],
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        
        'prefix-1': sentence[index][0],
        #'prefix-2': sentence[index][:2],
        #'prefix-3': sentence[index][:3],
        #'prefix-4': sentence[index][:4],
        'suffix-1': sentence[index][-1],
        #'suffix-2': sentence[index][-2:],
        #'suffix-3': sentence[index][-3:],

        'prev_word1': '' if index == 0 else sentence[index - 1],
        
        'next_word1': '' if index == len(sentence) - 1 else sentence[index + 1],        
        
        'is_numeric': sentence[index].isdigit(),
    }

In [ ]:
from nltk.tag.util import untag

def transform_to_dataset(tagged_sentences):
    X, y = [], []

    for tagged in tagged_sentences:
        try:
            X.append([features(untag(tagged), index) for index in range(len(tagged))])
            y.append([tag for _, tag in tagged])
        except ValueError:
            print(f"Problematic sentence: {tagged}")

    return X, y

X_train, y_train = transform_to_dataset(training_sentences)
X_test, y_test = transform_to_dataset(test_sentences)

print(len(X_train))
print(len(X_test))

1129
283


In [ ]:
!pip install sklearn_crfsuite

import time
start_time = time.time()

from sklearn_crfsuite import CRF
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
 
crf = sklearn_crfsuite.CRF(algorithm='lbfgs',
    c1=0.01,
    c2=0.01,
    max_iterations=100000,
    all_possible_transitions=True)
crf.fit(X_train, y_train)

print("--- %s seconds ---" % (time.time() - start_time))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.8 MB/s eta 0:00:00
--- 38.61729884147644 seconds ---


In [ ]:
labels = list(crf.classes_)
labels

['N',
 'RP',
 'VBD',
 'CC',
 'IN',
 'VBP',
 'VB',
 'PUNC',
 'CD',
 'ABBREV',
 'FW',
 'JJ',
 'DT',
 'RB',
 'JJR',
 'WP',
 'PRP',
 'WRB',
 'VBN']

In [ ]:
y_pred = crf.predict(X_test)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
a = mlb.fit_transform([crf.classes_])
y_test_matrix = mlb.transform(y_test)
y_pred_matrix = mlb.transform(y_pred)

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test_matrix, y_pred_matrix, target_names=crf.classes_, digits=3))

              precision    recall  f1-score   support

           N      1.000     0.867     0.929        30
          RP      0.996     1.000     0.998       239
         VBD      0.981     1.000     0.990        52
          CC      1.000     0.989     0.994        88
          IN      1.000     0.800     0.889         5
         VBP      1.000     1.000     1.000       256
          VB      0.892     0.900     0.896       211
        PUNC      0.952     0.833     0.889        48
          CD      1.000     1.000     1.000       280
      ABBREV      1.000     0.977     0.989        44
          FW      1.000     1.000     1.000       283
          JJ      0.969     0.969     0.969        97
          DT      0.974     0.995     0.984       186
          RB      0.900     0.692     0.783        26
         JJR      0.912     0.864     0.887       132
          WP      0.500     0.176     0.261        17
         PRP      0.925     0.961     0.943       206
         WRB      0.900    

In [ ]:
import time
start_time = time.time()

from sklearn_crfsuite import metrics
y_pred = crf.predict(X_test)
print("Accuracy = ",metrics.flat_accuracy_score(y_test, y_pred))
print ("Recall =", metrics.flat_recall_score(y_test, y_pred, average='weighted',  labels=labels))
print ("Precision =", metrics.flat_precision_score(y_test, y_pred, average='weighted',  labels=labels))
print ("F1 =", metrics.flat_f1_score(y_test, y_pred, average='weighted',  labels=labels))

print("--- %s seconds ---" % (time.time() - start_time))

Accuracy =  0.9262727272727272
Recall = 0.9262727272727272
Precision = 0.9241297653976086
F1 = 0.923770190122457
--- 0.19359421730041504 seconds ---


In [ ]:
y_pred = crf.predict(X_test)
print ("F1 =", metrics.flat_f1_score(y_test, y_pred, average='weighted',  labels=labels))

F1 = 0.923770190122457


In [ ]:
sentence = 'الله الرحمن الرحيم .'.split()
def pos_tag(sentence):
    sentence_features = [features(sentence, index) for index in range(len(sentence))]
    return list(zip(sentence, crf.predict([sentence_features])[0]))
 
print(pos_tag(sentence))

[('الله', 'N'), ('الرحمن', 'N'), ('الرحيم', 'JJ'), ('.', 'PUNC')]


# 1. 2. Save CRF

In [ ]:
import pickle
filename = 'crf_model.sav'
pickle.dump(crf, open(filename, 'wb'))

In [ ]:
import joblib
# Fit the CRF model
crf.fit(X_train, y_train)
# Save the CRF model
joblib.dump(crf, '/content/crf_model.sav')

['/content/crf_model.sav']

In [ ]:
import pickle

# Save the CRF model
with open('/content/crf_model.pkl', 'wb') as f:
    pickle.dump(crf, f)

# Load the CRF model
with open('/content/crf_model.pkl', 'rb') as f:
    crf_model = pickle.load(f)

# 2. 1. LSTM(RNN)

In [ ]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation
from tensorflow.keras.optimizers import Adam

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Read the dataset
dataset = pd.read_csv("/content/Talafeef.csv", usecols=['token', 'tpos'])

# Process the dataset to extract sentences and tags
sentences, tags = [], []
sentence, tag = [], []
for _, row in dataset.iterrows():
    if row["token"] == '.':
        sentences.append(sentence)
        tags.append(tag)
        sentence, tag = [], []
    else:
        sentence.append(row["token"])
        tag.append(row["tpos"])

# Add the last sentence if it doesn't end with a period
if len(sentence) > 0:
    sentences.append(sentence)
    tags.append(tag)

# Print the number of sentences
print("Number of sentences:", len(sentences))

# Get the first sentence and the first word and tag
if len(sentences) > 0 and len(sentences[0]) > 0:
    first_sentence = sentences[0]
    first_word, first_tag = first_sentence[0], tags[0][0]
    print("First word:", first_word)
    print("First tag:", first_tag)
else:
    print("No sentences found in the dataset.")


Number of sentences: 1412
First word: إذا
First tag: RP


In [ ]:
sentences, sentence_tags = [], []
sentence, tags = [], []
for index, row in dataset.iterrows():
    sentence.append(row['token'])
    tags.append(row['tpos'])
    if row['token'] == '.':
        sentences.append(np.array(sentence))
        sentence_tags.append(np.array(tags))
        sentence, tags = [], []

In [ ]:
sentences[2]

array(['و', 'حاليا', 'ف', 'إن', 'معظم', 'المرضى', 'غير', 'المستقرين',
       'الذين', 'يشك', 'لديهم', 'ب', 'استرواح', 'الصفاق', 'يخضعون',
       'مباشرة', 'ل', 'التصوير', 'المقطعي', 'المحوسب', '.'], dtype='<U9')

In [ ]:
sentence_tags[2]

array(['CC', 'N', 'CC', 'RP', 'JJ', 'N', 'N', 'JJ', 'WP', 'VBP', 'RB',
       'IN', 'N', 'N', 'VBP', 'JJ', 'IN', 'N', 'JJ', 'JJ', 'PUNC'],
      dtype='<U4')

In [ ]:
len(sentences[1]), len(sentence_tags[1000])

(26, 56)

In [ ]:
(train_sentences, 
 test_sentences, 
 train_tags, 
 test_tags) = train_test_split(sentences, sentence_tags, test_size=0.2)

In [ ]:
words, tags = set([]), set([])
 
for s in train_sentences:
    for w in s:
        words.add(w)
 
for ts in train_tags:
    for t in ts:
        tags.add(t)
 
word2index = {w: i + 2 for i, w in enumerate(list(words))}
word2index['-PAD-'] = 0  # The special value used for padding
word2index['-OOV-'] = 1  # The special value used for OOVs
 
tag2index = {t: i + 1 for i, t in enumerate(list(tags))}
tag2index['-PAD-'] = 0  # The special value used to padding

In [ ]:
len(word2index)

10379

In [ ]:
tag2index

{'VB': 1,
 'IN': 2,
 'JJ': 3,
 'VBN': 4,
 'JJR': 5,
 'PRP': 6,
 'VBD': 7,
 'CD': 8,
 'RB': 9,
 'DT': 10,
 'WRB': 11,
 'N': 12,
 'RP': 13,
 'ABBREV': 14,
 'VBP': 15,
 'CC': 16,
 'FW': 17,
 'PUNC': 18,
 'WP': 19,
 '-PAD-': 0}

In [ ]:
train_sentences_X, test_sentences_X, train_tags_y, test_tags_y = [], [], [], []
 
for s in train_sentences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    train_sentences_X.append(s_int)
 
for s in test_sentences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    test_sentences_X.append(s_int)
 
for s in train_tags:
    train_tags_y.append([tag2index[t] for t in s])
 
for s in test_tags:
    test_tags_y.append([tag2index[t] for t in s])

In [ ]:
print(train_sentences_X[0])
print(test_sentences_X[0])
print(train_tags_y[0])
print(test_tags_y[0])

[466, 5610, 3270, 5097, 5605, 4841, 1358, 9773, 9508, 4792, 4808, 9773, 9688, 2266, 466, 10083, 1905, 3718, 5648, 4519, 9241, 8076, 9640, 610, 3608, 2467, 4132, 6257, 466, 1128, 6257, 1358, 3385, 9042, 3609, 534, 1791, 7050, 10083, 9177, 610, 2452, 4807, 4682, 2499, 3385, 5648, 5616]
[4409, 10041, 466, 1, 10028, 3948, 185, 8982, 10091, 6573, 10091, 6406, 2709, 5668, 729, 10041, 4253, 3948, 1, 5533, 6062, 3014, 133, 6406, 8386, 1298, 1, 6406, 4505, 3168, 4253, 5123, 9390, 10083, 4038, 1, 2709, 3948, 1992, 8323, 4673, 3785, 303, 1, 10041, 3608, 129, 10083, 8310, 1085, 7210, 3948, 7420, 1, 5616]
[16, 13, 7, 12, 12, 12, 18, 2, 12, 10, 12, 2, 12, 12, 16, 2, 12, 15, 12, 2, 3, 12, 3, 2, 13, 15, 12, 3, 16, 12, 3, 18, 2, 12, 2, 12, 12, 12, 2, 12, 2, 12, 12, 12, 12, 2, 12, 18]
[12, 12, 16, 15, 12, 16, 15, 9, 12, 12, 12, 12, 12, 13, 15, 12, 7, 16, 7, 3, 11, 6, 12, 12, 12, 16, 12, 12, 3, 7, 7, 16, 7, 2, 9, 12, 12, 16, 7, 2, 12, 2, 13, 1, 12, 13, 2, 11, 12, 12, 11, 16, 13, 15, 18]


In [ ]:
MAX_LENGTH = max([len(s) for s in train_sentences_X])
MAX_LENGTH

3105

In [ ]:
train_sentences_X = pad_sequences(train_sentences_X, maxlen=MAX_LENGTH, padding='post')
test_sentences_X = pad_sequences(test_sentences_X, maxlen=MAX_LENGTH, padding='post')
train_tags_y = pad_sequences(train_tags_y, maxlen=MAX_LENGTH, padding='post')
test_tags_y = pad_sequences(test_tags_y, maxlen=MAX_LENGTH, padding='post')

In [ ]:
len(train_sentences_X[0]),len(train_sentences_X[5])

(3105, 3105)

In [ ]:
model = Sequential()
model.add(InputLayer(input_shape=(MAX_LENGTH, )))
model.add(Embedding(len(word2index), 128))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(TimeDistributed(Dense(len(tag2index))))
model.add(Activation('softmax'))
 
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3105, 128)         1328512   
                                                                 
 bidirectional (Bidirectiona  (None, 3105, 512)        788480    
 l)                                                              
                                                                 
 time_distributed (TimeDistr  (None, 3105, 20)         10260     
 ibuted)                                                         
                                                                 
 activation (Activation)     (None, 3105, 20)          0         
                                                                 
Total params: 2,127,252
Trainable params: 2,127,252
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def to_categorical(sequences, categories):
    cat_sequences = []
    for s in sequences:
        cats = []
        for item in s:
            cats.append(np.zeros(categories))
            cats[-1][item] = 1.0
        cat_sequences.append(cats)
    return np.array(cat_sequences)

In [ ]:
cat_train_tags_y = to_categorical(train_tags_y, len(tag2index))

In [ ]:
cat_train_tags_y.shape

(1129, 3105, 20)

In [ ]:
model.fit(train_sentences_X, to_categorical(train_tags_y, len(tag2index)), batch_size=128, epochs=40, validation_split=0.2)

Epoch 1/40
8/8 [==============================] - 17s 1s/step - loss: 1.6890 - accuracy: 0.8473 - val_loss: 0.1465 - val_accuracy: 0.9845
Epoch 2/40
8/8 [==============================] - 7s 922ms/step - loss: 0.1345 - accuracy: 0.9861 - val_loss: 0.1111 - val_accuracy: 0.9845
Epoch 3/40
8/8 [==============================] - 7s 883ms/step - loss: 0.0563 - accuracy: 0.9871 - val_loss: 0.0473 - val_accuracy: 0.9879
Epoch 4/40
8/8 [==============================] - 7s 930ms/step - loss: 0.0409 - accuracy: 0.9891 - val_loss: 0.0395 - val_accuracy: 0.9896
Epoch 5/40
8/8 [==============================] - 7s 892ms/step - loss: 0.0366 - accuracy: 0.9901 - val_loss: 0.0368 - val_accuracy: 0.9902
Epoch 6/40
8/8 [==============================] - 8s 948ms/step - loss: 0.0328 - accuracy: 0.9913 - val_loss: 0.0397 - val_accuracy: 0.9903
Epoch 7/40
8/8 [==============================] - 7s 913ms/step - loss: 0.0317 - accuracy: 0.9912 - val_loss: 0.0391 - val_accuracy: 0.9902
Epoch 8/40
8/8 [======

In [ ]:
scores = model.evaluate(test_sentences_X, to_categorical(test_tags_y, len(tag2index)))
print(f"{model.metrics_names[1]}: {scores[1] * 100}")

9/9 [==============================] - 1s 128ms/step - loss: 0.0231 - accuracy: 0.9933
accuracy: 99.33493733406067


In [ ]:
test_samples = [
    "إذا تم إجراء صورة شعاعية ل البطن و صورة شعاعية ل الصدر ب وضعية الوقوف و مازال تشخيص استرواح الصفاق غير مؤكد ف عندها يجب طلب تصوير مقطعي محوسب ل البطن .".split(),
]
print(test_samples)

[['إذا', 'تم', 'إجراء', 'صورة', 'شعاعية', 'ل', 'البطن', 'و', 'صورة', 'شعاعية', 'ل', 'الصدر', 'ب', 'وضعية', 'الوقوف', 'و', 'مازال', 'تشخيص', 'استرواح', 'الصفاق', 'غير', 'مؤكد', 'ف', 'عندها', 'يجب', 'طلب', 'تصوير', 'مقطعي', 'محوسب', 'ل', 'البطن', '.']]


In [ ]:
test_samples_X = []
for s in test_samples:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
    test_samples_X.append(s_int)
 
test_samples_X = pad_sequences(test_samples_X, maxlen=MAX_LENGTH, padding='post')
print(test_samples_X)

[[9851 3270  992 ...    0    0    0]]


In [ ]:
predictions = model.predict(test_samples_X)
print(predictions, predictions.shape)

1/1 [==============================] - 1s 720ms/step
[[[2.4239480e-02 6.7803590e-03 1.4114192e-01 ... 2.4427839e-03
   5.8488343e-02 8.2946522e-03]
  [2.1790367e-02 6.2090796e-03 1.3488978e-01 ... 2.2918703e-03
   7.0508882e-02 7.7144867e-03]
  [1.8491639e-02 5.2619022e-03 9.8328963e-02 ... 1.9654343e-03
   5.9428673e-02 6.4718835e-03]
  ...
  [9.9999106e-01 3.9185534e-07 4.0105324e-08 ... 5.7122918e-07
   2.9324117e-06 1.9954112e-07]
  [9.9998665e-01 6.2608416e-07 5.3281596e-08 ... 9.0568295e-07
   3.7275167e-06 3.1438282e-07]
  [9.9998164e-01 9.0795635e-07 6.8095623e-08 ... 1.2972095e-06
   4.5859420e-06 4.4967894e-07]]] (1, 3105, 20)


In [ ]:
def logits_to_tokens(sequences, index):
    token_sequences = []
    for categorical_sequence in sequences:
        token_sequence = []
        for categorical in categorical_sequence:
            token_sequence.append(index[np.argmax(categorical)])
 
        token_sequences.append(token_sequence)
 
    return token_sequences

In [ ]:
print(logits_to_tokens(predictions, {i: t for t, i in tag2index.items()})[0])

['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'PUNC', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-',

In [ ]:
from keras import backend as K
 
def ignore_class_accuracy(to_ignore=0):
    def ignore_accuracy(y_true, y_pred):
        y_true_class = K.argmax(y_true, axis=-1)
        y_pred_class = K.argmax(y_pred, axis=-1)
 
        ignore_mask = K.cast(K.not_equal(y_pred_class, to_ignore), 'int32')
        matches = K.cast(K.equal(y_true_class, y_pred_class), 'int32') * ignore_mask
        accuracy = K.sum(matches) / K.maximum(K.sum(ignore_mask), 1)
        return accuracy
    return ignore_accuracy

In [ ]:
model = Sequential()
model.add(InputLayer(input_shape=(MAX_LENGTH, )))
model.add(Embedding(len(word2index), 128))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(TimeDistributed(Dense(len(tag2index))))
model.add(Activation('softmax'))
 
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy', ignore_class_accuracy(0)])
 
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 3105, 128)         1328512   
                                                                 
 bidirectional_1 (Bidirectio  (None, 3105, 512)        788480    
 nal)                                                            
                                                                 
 time_distributed_1 (TimeDis  (None, 3105, 20)         10260     
 tributed)                                                       
                                                                 
 activation_1 (Activation)   (None, 3105, 20)          0         
                                                                 
Total params: 2,127,252
Trainable params: 2,127,252
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_sentences_X, to_categorical(train_tags_y, len(tag2index)), batch_size=128, epochs=40, validation_split=0.2)

Epoch 1/40
8/8 [==============================] - 12s 1s/step - loss: 1.7106 - accuracy: 0.8463 - ignore_accuracy: 0.0031 - val_loss: 0.1550 - val_accuracy: 0.9845 - val_ignore_accuracy: 0.0000e+00
Epoch 2/40
8/8 [==============================] - 8s 951ms/step - loss: 0.1390 - accuracy: 0.9861 - ignore_accuracy: 0.0000e+00 - val_loss: 0.1267 - val_accuracy: 0.9845 - val_ignore_accuracy: 0.0000e+00
Epoch 3/40
8/8 [==============================] - 7s 921ms/step - loss: 0.0573 - accuracy: 0.9861 - ignore_accuracy: 0.0104 - val_loss: 0.0430 - val_accuracy: 0.9845 - val_ignore_accuracy: 0.2000
Epoch 4/40
8/8 [==============================] - 8s 1s/step - loss: 0.0380 - accuracy: 0.9895 - ignore_accuracy: 0.3357 - val_loss: 0.0378 - val_accuracy: 0.9904 - val_ignore_accuracy: 0.4082
Epoch 5/40
8/8 [==============================] - 7s 929ms/step - loss: 0.0339 - accuracy: 0.9909 - ignore_accuracy: 0.3826 - val_loss: 0.0355 - val_accuracy: 0.9904 - val_ignore_accuracy: 0.4098
Epoch 6/40
8/

In [ ]:
predictions = model.predict(test_sentences_X)
print(logits_to_tokens(predictions, {i: t for t, i in tag2index.items()}))

9/9 [==============================] - 2s 120ms/step


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
from sklearn.metrics import accuracy_score

# Evaluate the model
scores = model.evaluate(test_sentences_X, to_categorical(test_tags_y, len(tag2index)))
print(f"{model.metrics_names[1]}: {scores[1] * 100}")

# Make predictions
predictions = model.predict(test_sentences_X)

# Convert the categorical predictions to token sequences
predicted_tags = logits_to_tokens(predictions, {i: t for t, i in tag2index.items()})

# Flatten the true and predicted tags for the test set
true_tags_1d = [tag for tags_sequence in test_tags_y for tag in tags_sequence]
predicted_tags_1d = [tag for tags_sequence in predicted_tags for tag in tags_sequence]

# Calculate and print the overall accuracy
overall_accuracy = accuracy_score(true_tags_1d, predicted_tags_1d)
print(f"Overall Accuracy: {overall_accuracy:.2f}")

9/9 [==============================] - 1s 131ms/step - loss: 0.0239 - accuracy: 0.9933 - ignore_accuracy: 0.4855
accuracy: 99.33448433876038
9/9 [==============================] - 1s 119ms/step
Overall Accuracy: 0.00


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:227: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred


# 2. 2. Save LSTM (RNN)

In [ ]:
# Save Model
model.save("RNN_model.h5")

In [ ]:
import pickle
a_file = open("RNN_tag2index.pkl", "wb")
pickle.dump(tag2index, a_file)
a_file.close()

In [ ]:
b_file = open("RNN_word2index.pkl", "wb")
pickle.dump(word2index, b_file)
b_file.close()

# 3. 1. Word2Vec (Skip-Gram Model)

In [ ]:
#!pip install --upgrade allennlp
!pip install allennlp==2.5.0
!pip install google-cloud-storag
!pip install overrides
!pip install python-bidi
!pip install arabic_reshaper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.4/681.4 kB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 804.1/804.1 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.2/594.2 kB 54.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 25.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
import math
import random
from collections import Counter
import numpy as np
import torch
import torch.optim as optim
from allennlp.common.file_utils import cached_path
from allennlp.data.data_loaders import SimpleDataLoader
from allennlp.data.dataset_readers.dataset_reader import DatasetReader
from allennlp.data.fields import LabelField
from allennlp.data.instance import Instance
from allennlp.data.vocabulary import Vocabulary
from allennlp.models import Model
from allennlp.modules.token_embedders import Embedding
from allennlp.training import GradientDescentTrainer
from overrides import overrides
from scipy.stats import spearmanr
from torch.nn import CosineSimilarity
from torch.nn import functional
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import matplotlib.pyplot as plt
%matplotlib inline
from bidi.algorithm import get_display
import arabic_reshaper

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
@DatasetReader.register("skip_gram")
class SkipGramReader(DatasetReader):
    def __init__(self, window_size=5, vocab: Vocabulary=None):
        """A DatasetReader for reading a plain text corpus and producing instances
        for the SkipGram model.
        When vocab is not None, this runs sub-sampling of frequent words as described
        in (Mikolov et al. 2013).
        """
        super().__init__()
        self.window_size = window_size
        self.reject_probs = None
        if vocab:
            self.reject_probs = {}
            threshold = 1.e-3
            token_counts = vocab._retained_counter['token_in']  # HACK
            total_counts = sum(token_counts.values())
            for _, token in vocab.get_index_to_token_vocabulary('token_in').items():
                counts = token_counts[token]
                if counts > 0:
                    normalized_counts = counts / total_counts
                    reject_prob = 1. - math.sqrt(threshold / normalized_counts)
                    reject_prob = max(0., reject_prob)
                else:
                    reject_prob = 0.
                self.reject_probs[token] = reject_prob

    def _subsample_tokens(self, tokens):
        """Given a list of tokens, runs sub-sampling.
        Returns a new list of tokens where rejected tokens are replaced by Nones.
        """
        new_tokens = []
        for token in tokens:
            reject_prob = self.reject_probs.get(token, 0.)
            if random.random() <= reject_prob:
                new_tokens.append(None)
            else:
                new_tokens.append(token)

        return new_tokens

    @overrides
    def _read(self, file_path: str):
        with open(cached_path(file_path), "r") as text_file:
            for line in text_file:
                tokens = line.strip().split(' ')
                tokens = tokens[:1000000]  # TODO: remove

                if self.reject_probs:
                    tokens = self._subsample_tokens(tokens)
                    print(tokens[:200])  # for debugging

                for i, token in enumerate(tokens):
                    if token is None:
                        continue

                    token_in = LabelField(token, label_namespace='token_in')

                    for j in range(i - self.window_size, i + self.window_size + 1):
                        if j < 0 or i == j or j > len(tokens) - 1:
                            continue

                        if tokens[j] is None:
                            continue

                        token_out = LabelField(tokens[j], label_namespace='token_out')
                        yield Instance({'token_in': token_in, 'token_out': token_out})

In [ ]:
EMBEDDING_DIM = 256
BATCH_SIZE = 256
CUDA_DEVICE = -1

In [ ]:
class SkipGramModel(Model):
    def __init__(self, vocab, embedding_in):
        super().__init__(vocab)
        self.embedding_in = embedding_in
        self.linear = torch.nn.Linear(
            in_features=EMBEDDING_DIM,
            out_features=vocab.get_vocab_size('token_out'),
            bias=False)

    def forward(self, token_in, token_out):
        embedded_in = self.embedding_in(token_in)
        logits = self.linear(embedded_in)
        loss = functional.cross_entropy(logits, token_out)

        return {'loss': loss}

In [ ]:
def get_related(token: str, embedding: Model, vocab: Vocabulary, num_synonyms: int = 10):
    """Given a token, return a list of top N most similar words to the token."""
    token_id = vocab.get_token_index(token, 'token_in')
    token_vec = embedding.weight[token_id]
    cosine = CosineSimilarity(dim=0)
    sims = Counter()

    for index, token in vocab.get_index_to_token_vocabulary('token_in').items():
        sim = cosine(token_vec, embedding.weight[index]).item()
        sims[token] = sim

    return sims.most_common(num_synonyms)

In [ ]:
reader = SkipGramReader()
talafeef = reader.read("/content/Talafeef--Seg.txt")

In [ ]:
talafeef = list(talafeef)
print(len(talafeef))

591610


In [ ]:
vocab = Vocabulary.from_instances(
    talafeef, min_count={'token_in': 2, 'token_out': 2},max_vocab_size=100000)

building vocab: 100%|##########| 591610/591610 [00:01<00:00, 389733.30it/s]


In [ ]:
data_loader = SimpleDataLoader(talafeef, batch_size=BATCH_SIZE)
data_loader.index_with(vocab)

In [ ]:
vocab.get_vocab_size

<bound method Vocabulary.get_vocab_size of Vocabulary with namespaces:  token_in, Size: 11421 || token_out, Size: 11421 || Non Padded Namespaces: {'*labels', '*tags'}>

In [ ]:
embedding_in = Embedding(num_embeddings=vocab.get_vocab_size('token_in'),
                         embedding_dim=EMBEDDING_DIM)

In [ ]:
model = SkipGramModel(vocab=vocab,
                      embedding_in=embedding_in)

In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
trainer = GradientDescentTrainer(
    model=model,
    optimizer=optimizer,
    data_loader=data_loader,
    num_epochs=5,
    cuda_device=-1)

In [ ]:
trainer.train()

batch_loss: 8.8472, loss: 8.2816 ||: 100%|##########| 2311/2311 [07:49<00:00,  4.92it/s]
batch_loss: 8.4069, loss: 7.2477 ||: 100%|##########| 2311/2311 [07:31<00:00,  5.12it/s]
batch_loss: 8.1822, loss: 7.0402 ||: 100%|##########| 2311/2311 [07:37<00:00,  5.05it/s]
batch_loss: 7.8415, loss: 6.8903 ||: 100%|##########| 2311/2311 [07:59<00:00,  4.82it/s]
batch_loss: 7.6669, loss: 6.6898 ||: 100%|##########| 2311/2311 [07:50<00:00,  4.91it/s]


{'best_epoch': 4,
 'peak_worker_0_memory_MB': 1235.10546875,
 'peak_gpu_0_memory_MB': 0,
 'training_duration': '0:38:49.447625',
 'epoch': 4,
 'training_loss': 6.689805425745437,
 'training_worker_0_memory_MB': 1235.10546875,
 'training_gpu_0_memory_MB': 0.0}

# 3. 2. Save Skip-Gram Model

In [ ]:
torch.save(model, r"SkipGram_model.pt")
#with open("model.th", 'wb') as f:
#    torch.save(model.state_dict(), f)
#trainer.save('model.pt')

In [ ]:
print(get_related('العلوم', embedding_in, vocab))

[('العلوم', 1.0), ('الممارسة', 0.8805922865867615), ('التاريخي', 0.8782991766929626), ('البعد', 0.873658299446106), ('العلاقة', 0.8732448220252991), ('العقلاني', 0.8651537299156189), ('دراسات', 0.8614330887794495), ('إدراك', 0.8558136224746704), ('سرقة', 0.8554249405860901), ('أسماء', 0.8533512353897095)]


In [ ]:
print(get_related('محمد', embedding_in, vocab))

[('محمد', 1.0), ('بارك', 0.820411741733551), ('آله', 0.7994131445884705), ('رسولك', 0.7942121624946594), ('إبراهيم', 0.76529860496521), ('عبدك', 0.7583552598953247), ('نبينا', 0.7350561022758484), ('صل', 0.7037534713745117), ('صلاة', 0.7007614374160767), ('أصحابه', 0.6974852085113525)]


In [ ]:
print(get_related('السعودية', embedding_in, vocab))

[('السعودية', 1.0), ('العربية', 0.8396695852279663), ('المملكة', 0.8040644526481628), ('ملك', 0.7038750648498535), ('عبدالعزيز', 0.6806778311729431), ('سعود', 0.6583635210990906), ('الحكم', 0.6306524872779846), ('آل', 0.627746045589447), ('الرسمية', 0.6237347722053528), ('الاطلاع', 0.6213890314102173)]


In [ ]:
print(get_related('النظام', embedding_in, vocab))

[('النظام', 1.0), ('أحكام', 0.7862368822097778), ('تعديل', 0.7843925356864929), ('الرسمية', 0.7344747185707092), ('الأساسي', 0.7308825254440308), ('الجريدة', 0.7304423451423645), ('اللائحة', 0.7206048369407654), ('تعدل', 0.7172296643257141), ('اختصاصاته', 0.705812394618988), ('ثالثا', 0.6978699564933777)]


In [ ]:
print(get_related('الشباب', embedding_in, vocab))

[('الشباب', 1.0), ('مهارات', 0.8578124046325684), ('المهارة', 0.850523829460144), ('الوقوع', 0.8404920101165771), ('سوق', 0.8345710635185242), ('الطاقة', 0.83315509557724), ('مطلوبة', 0.8318454623222351), ('القطاعين', 0.8315698504447937), ('الموجودة', 0.8243515491485596), ('جميعها', 0.823792040348053)]


In [ ]:
token_id = vocab.get_token_index("الله", 'token_in')
token_vec = embedding_in.weight[token_id]

In [ ]:
cosine = CosineSimilarity(dim=0)

In [ ]:
vocab.get_vocab_size

<bound method Vocabulary.get_vocab_size of Vocabulary with namespaces:  token_in, Size: 11421 || token_out, Size: 11421 || Non Padded Namespaces: {'*labels', '*tags'}>

In [ ]:
import numpy as np
np.unique(np.array(talafeef))

array(['token_in', 'token_out'], dtype='<U9')

In [ ]:
len(talafeef)

591610

In [ ]:
vocab.get_vocab_size

<bound method Vocabulary.get_vocab_size of Vocabulary with namespaces:  token_in, Size: 11421 || token_out, Size: 11421 || Non Padded Namespaces: {'*labels', '*tags'}>

In [ ]:
vocab.get_token_to_index_vocabulary('token_in').items()

dict_items([('@@PADDING@@', 0), ('@@UNKNOWN@@', 1), ('و', 2), ('،', 3), ('ب', 4), ('ل', 5), ('.', 6), ('في', 7), ('من', 8), ('على', 9), ('أن', 10), ('الله', 11), ('ف', 12), ('ما', 13), ('لا', 14), ('إلى', 15), ('المادة', 16), ('عن', 17), ('أو', 18), ('هذا', 19), ('التي', 20), ('ذلك', 21), ('مجلس', 22), ('إن', 23), ('كل', 24), ('هو', 25), ('هذه', 26), ('عليه', 27), ('مع', 28), ('الصورة', 29), ('الأمعاء', 30), ('اللهم', 31), ('الذي', 32), ('بعد', 33), ('قد', 34), ('النظام', 35), ('تظهر', 36), ('الوزراء', 37), ('إلا', 38), ('له', 39), ('هي', 40), ('الملك', 41), ('سلم', 42), ('قال', 43), ('كما', 44), ('إذا', 45), ('كان', 46), ('رئيس', 47), ('صلى', 48), ('بين', 49), ('يكون', 50), ('ك', 51), ('المجلس', 52), ('أكثر', 53), ('الدولة', 54), ('أي', 55), ('حيث', 56), ('شكل', 57), ('المنطقة', 58), ('الهيئة', 59), ('غير', 60), ('أنه', 61), ('البطن', 62), ('الشعاعية', 63), ('تكون', 64), ('الذين', 65), ('يا', 66), ('لم', 67), ('العمل', 68), ('خلال', 69), ('عشرة', 70), ('الغاز', 71), ('عليها', 72), ('ح

In [ ]:
vocab.get_vocab_size("token_out")

11421

In [ ]:
embedding_in.weight.shape

torch.Size([11421, 256])

# 4. 1. Word2Vec (CBOW model)

In [ ]:
import numpy as np
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from collections import Counter

In [ ]:
data=open("/content/Talafeef--Seg.txt",'r')
talafeef_data = [text for text in data if text.count(' ') >= 2]

In [ ]:
talafeef_data[0]

' إذا تم إجراء صورة شعاعية ل البطن و صورة شعاعية ل الصدر ب وضعية الوقوف و مازال تشخيص استرواح الصفاق غير مؤكد ف عندها يجب طلب تصوير مقطعي محوسب ل البطن . يعرض التصوير المقطعي المحوسب المريض ل كمية أكبر من الإشعاع لكنه سيظهر ب وضوح وجود الغاز السهم الأبيض و قد يشخص السبب المستبطن . و حاليا ف إن معظم المرضى غير المستقرين الذين يشك لديهم ب استرواح الصفاق يخضعون مباشرة ل التصوير المقطعي المحوسب . مثال مثال الصورة صورتان شعاعيتان ل البطن متطابقتان تظهران استرواح صفاق كبير . يوجد عرى معوية يحددها خارجيا غاز من كلا الجانبين ب شكل يتوافق مع علامة . تظهر الصورة اليمنى المناطق التي تكون فيها علامة أكثر وضوحا ب لون بني و فيروزي لمعة الأمعاء معلمة ب لون بني أما الغاز الحر الذي يحدد جدار الأمعاء خارجيا معلم ب لون فيروزي صورتان شعاعيتان متطابقتان تظهران استرواح صفاق كبير . يوجد عرى معوية يحددها خارجيا غاز من كلا الجانبين ب شكل يتوافق مع علامة تظهر الصورة اليمنى المناطق حيث يكون استرواح الصفاق أكثر وضوحا ب اللون الفيروزي و لمعة الأمعاء معلمة ب اللون البني . يمكنك أيضا أن ترى الغاز يحدد الكبد خارجيا ك

In [ ]:
vectorize = Tokenizer()
vectorize.fit_on_texts(talafeef_data)
talafeef_data = vectorize.texts_to_sequences(talafeef_data)
total_vocab = sum(len(s) for s in talafeef_data)
word_count = len(vectorize.word_index) + 1
window_size = 2

In [ ]:
word_count

11415

In [ ]:
total_vocab

57741

In [ ]:
!pip install --upgrade tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.2/178.2 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: google-auth
    Found existing installation: google-auth 1.35.0
    Uninstalling google-auth-1.35.0:
      Successfully uninstalled google-auth-1.35.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-storage 1.38.0 requires google-auth<2.0dev,>=1.11.0, but you have google-auth 2.17.3 which is incompatible.
google-cloud-core 1.7.3 requires google-auth<2.0dev,>=1.24.0, but you have google-auth 2.17.3 which is incompatible.


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding, Lambda
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K

In [ ]:
def cbow_model(data, window_size, total_vocab):
    total_length = window_size*2
    for text in data:
        text_len = len(text)
        for idx, word in enumerate(text):
            context_word = []
            target   = []            
            begin = idx - window_size
            end = idx + window_size + 1
            context_word.append([text[i] for i in range(begin, end) if 0 <= i < text_len and i != idx])
            target.append(word)
            contextual = pad_sequences(context_word, maxlen=total_length)
            final_target = np_utils.to_categorical(target, total_vocab)
            yield(contextual, final_target)

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=total_vocab, output_dim=100, input_length=window_size*2))
model.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(100,)))
model.add(Dense(total_vocab, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
for i in range(10):
    cost = 0
    for x, y in cbow_model(talafeef_data, window_size, total_vocab):
        cost += model.train_on_batch(x, y)
    print(i, cost)

0 511941.76701672375
1 465041.3886981467
2 418825.4199489064
3 372395.67551187775
4 328930.94624006166
5 290391.46792476973
6 257724.52018575402
7 229016.75827746984
8 203964.11474394135
9 182369.23941060208


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 100)            5774100   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 57741)             5831841   
                                                                 
Total params: 11,605,941
Trainable params: 11,605,941
Non-trainable params: 0
_________________________________________________________________


# 4. 2. Save CBOW Model 

In [ ]:
model.save("cbow_model.h5")

In [ ]:
weights = model.get_weights()[0]
w2v_my = {}

for word, index in vectorize.word_index.items():
    w2v_my[word] = weights[index]

np.savez('embeddings.npz', **w2v_my)
embeddings = np.load('embeddings.npz')

In [ ]:
embeddings["الله"]

array([ 0.27993688,  0.07120431,  0.26767626,  0.25282174,  1.0951012 ,
        1.6889765 , -0.20246087,  1.7091613 , -0.7632016 ,  1.120734  ,
       -0.4092044 ,  0.5716477 , -0.29057154, -0.08226096, -0.42095208,
        0.08825853,  0.18974511, -0.08490095,  0.7360795 ,  0.14302415,
        0.7060526 ,  1.6797556 , -0.5505946 ,  1.7666436 , -0.21437186,
       -0.63364285,  0.47310987, -0.22869018,  0.29482394,  0.49904194,
       -0.21884209, -1.3027201 , -0.32482287, -1.746755  ,  1.1081969 ,
       -0.1649091 ,  0.3653062 ,  0.18415271, -0.93456995,  0.7247719 ,
       -0.5408487 , -0.7594056 ,  0.672341  , -0.1186127 , -0.5444066 ,
       -0.22125424, -0.6010884 ,  0.60653484,  0.84577733,  0.14000566,
        1.457906  , -0.503109  ,  0.39016327,  0.7339899 , -0.09890876,
        0.52188367,  0.08965507, -0.66726094,  1.3173606 , -0.5732715 ,
       -1.1861007 ,  0.5143744 ,  0.46508154,  1.6168398 , -1.7130787 ,
       -0.691999  ,  0.24741264,  0.39214978, -0.03980275, -0.13

In [ ]:
def get_related(token: str, num_synonyms: int = 10):
    """Given a token, return a list of top N most similar words to the token."""
    token_vec = w2v_my[token]
    sims = Counter()

    for w,vec in w2v_my.items():
        sim = np.dot(token_vec, vec)/(np.linalg.norm(token_vec) * np.linalg.norm(vec))
        sims[w] = sim

    return sims.most_common(num_synonyms)

In [ ]:
get_related("أشعة",10)

[('أشعة', 1.0000001),
 ('موجات', 0.5669669),
 ('رسم', 0.4941221),
 ('صوتية', 0.46655378),
 ('يوصف', 0.44993737),
 ('تفرج', 0.4473002),
 ('صالحا', 0.43631655),
 ('أجلهما', 0.43050963),
 ('مزيدا', 0.42970288),
 ('الطبية', 0.42583993)]

In [ ]:
get_related("الهندسة",10)

[('الهندسة', 1.0000001),
 ('أبل', 0.5488208),
 ('نمو', 0.5333013),
 ('صفها', 0.51157445),
 ('الحديثة', 0.5011609),
 ('الثقافية', 0.49718523),
 ('العنصرية', 0.49681476),
 ('الطيران', 0.49663526),
 ('أدوات', 0.48633814),
 ('إبراز', 0.48501801)]

In [ ]:
np.array(list(w2v_my.values())).shape

(11414, 100)

# 5.1. araBERTv02 (pre-trained model)

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 32.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
import logging
import glob
import os

In [3]:
! curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
! sudo apt-get install git-lfs
! git lfs install
!  clone https://huggingface.co/aubmindlab/bert-base-arabertv2
! tar -C ./MODEL_NAME -zxvf /content/bert-base-arabertv02/tf1_model.tar.gz

Detected operating system as Ubuntu/focal.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Detected apt version as 2.0.9
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... Packagecloud gpg key imported to /etc/apt/keyrings/github_git-lfs-archive-keyring.gpg
done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages will be upgraded:
  git-lfs
1 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Need to get 7,419 kB of archives.
After this operation, 4,936 kB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu focal/main amd64 git-lfs amd64 3.3.0 [7,419 kB]
Fetched 7,419 kB in 1s (6,517 kB/s)
debconf: unable to initialize frontend: Dialog
debcon

In [11]:
! pip install pyarabic
!pip install farasapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
! git clone https://github.com/aub-mind/arabert.git

fatal: destination path 'arabert' already exists and is not an empty directory.


In [13]:
from arabert.preprocess import ArabertPreprocessor
model_name = "aubmindlab/bert-base-arabertv2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

arabert_prep = ArabertPreprocessor(model_name=model_name)

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv2 were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTT

100%|██████████| 241M/241M [00:18<00:00, 13.3MiB/s]


[2023-04-29 14:54:27,144 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [8]:
with open("/content/Talafeef--Seg.txt",'r', encoding='utf-8') as f:
    dataset_tokens = f.read()

tokens = dataset_tokens.split(" ")
number_of_tokens = len(tokens)

In [14]:
preprocessed_text = arabert_prep.preprocess(dataset_tokens)

In [15]:
preprocessed_text.split(".")[0]

'إذا تم إجراء صور +ة شعاعي +ة ل ال+ بطن و صور +ة شعاعي +ة ل ال+ صدر ب وضعي +ة ال+ وقوف و مازال تشخيص استرواح ال+ صفاق غير مؤكد ف عند +ها يجب طلب تصوير مقطع +ي محوسب ل ال+ بطن '

In [16]:
splitted_text = preprocessed_text.split(".")
marked_text=''
for sent in splitted_text:
  marked_text = marked_text +  "[CLS] " + sent + " [SEP]"

tokenized_text = tokenizer.tokenize(marked_text)
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

segments_ids=[]
id=0
for token in tokenized_text:
  if token != '[CLS]':
    segments_ids.append(id)
  else:
    segments_ids.append(id)
    id = id + 1
print(len(indexed_tokens))

Token indices sequence length is longer than the specified maximum sequence length for this model (92237 > 512). Running this sequence through the model will result in indexing errors


92237


In [17]:
tokenized_text[:10]

['[CLS]', 'إذا', 'تم', 'إجراء', 'صور', '+ة', 'شعاعي', '+ة', 'ل', 'ال+']

In [18]:
len(segments_ids), len(tokenized_text)

(92237, 92237)

In [19]:
indexed_tokens[:10], segments_ids[:10],tokenized_text[:10]

([33, 985, 408, 1061, 947, 12, 25050, 12, 162, 20],
 [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 ['[CLS]', 'إذا', 'تم', 'إجراء', 'صور', '+ة', 'شعاعي', '+ة', 'ل', 'ال+'])

In [20]:
def bert_processing (model, segments_ids, indexed_tokens, tokens_in_chunk):
		
	tensor_input_ids = torch.tensor(indexed_tokens).unsqueeze(0)
	segments_tensor = torch.tensor(segments_ids).unsqueeze(0)
	output = model(tensor_input_ids,segments_tensor)

	hidden_states = output[0][0]

	token_vecs_sum = []

	for token in hidden_states.detach().numpy():
		token_vecs_sum.append(token)

		# `token` is a [12 x 768] tensor

		# Sum the vectors from the last four layers.
		#sum_vec = torch.sum(token[-4:], dim=0)
		
		# Use `sum_vec` to represent `token`.
		#token_vecs_sum.append(sum_vec)

	#print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))


	token_vecs_cat_array=np.empty((tokens_in_chunk,768),float)
	for t in token_vecs_sum:
	  np.append(token_vecs_cat_array,np.array(t))
	
	#token_vecs_cat_array.shape
	return token_vecs_cat_array

In [21]:
final_token_vecs_array = np.empty((len(indexed_tokens),768),float)

for i in range(0,len(segments_ids),512):
    x=i
    chunk_segments_ids = segments_ids[x:x+512]
    chunk_indexed_tokens = indexed_tokens[x:x+512]
    chunk_token_vecs = bert_processing(model=model, segments_ids=chunk_segments_ids,
                                       indexed_tokens=chunk_indexed_tokens, 
                                       tokens_in_chunk=len(chunk_indexed_tokens))
    #np.append(final_token_vecs_array, np.array(chunk_token_vecs))
    final_token_vecs_array[x:x+512] = chunk_token_vecs

In [22]:
final_token_vecs_array.shape

(92237, 768)

In [23]:
from collections import Counter
def get_related(token: str, num_synonyms: int = 10, 
                tokenized_text = tokenized_text ,final_token_vecs_array=final_token_vecs_array):
    """Given a token, return a list of top N most similar words to the token."""
    token_vec = final_token_vecs_array[tokenized_text.index(token)]
    sims = Counter()

    for i in range (final_token_vecs_array.shape[0]):
        sim = np.dot(token_vec, final_token_vecs_array[i])/(np.linalg.norm(token_vec) * np.linalg.norm(final_token_vecs_array[i]))
        sims[tokenized_text[i]] = sim

    return sims.most_common(num_synonyms)

# 5.2. Save araBERT (Embeddings)  

In [24]:
#save token vecs
from numpy import asarray
from numpy import savez_compressed 
savez_compressed('token_vecs_cat_array.npz', final_token_vecs_array)

In [25]:
#test if saved token vec saving done right (it should return array([[ True,  True,  True, ..., )
from numpy import load
token_vecs_cat_array2 = load('token_vecs_cat_array.npz')
token_vecs_cat_array2==final_token_vecs_array

data= load('token_vecs_cat_array.npz')
token_vecs_cat_array2 = data['arr_0']  
token_vecs_cat_array2==final_token_vecs_array

<ipython-input-25-bf4e930dbb38>:4: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  token_vecs_cat_array2==final_token_vecs_array


array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [26]:
#save tokenized text
import pickle
open_file = open("new_tokenized_text.pkl", "wb")
pickle.dump(tokenized_text, open_file)
open_file.close()

In [27]:
#check if tokenized text saving done right (should return True)
open_file = open("new_tokenized_text.pkl", "rb")
tokenized_text2 = pickle.load(open_file)
open_file.close()

tokenized_text2==tokenized_text 

True

In [28]:
#save tokenized vec (method 2)
import pickle
open_file = open("token_vecs_cat_array.pkl", "wb")
pickle.dump(final_token_vecs_array, open_file)
open_file.close() 

In [29]:
#check if tokenized text saving done right (method 2)
open_file = open("token_vecs_cat_array.pkl", "rb")
tokenized_vec2 = pickle.load(open_file)
open_file.close()

tokenized_vec2==final_token_vecs_array 

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [30]:
get_related("مقطع",10)

<ipython-input-23-21af80016e4b>:9: RuntimeWarning: invalid value encountered in double_scalars
  sim = np.dot(token_vec, final_token_vecs_array[i])/(np.linalg.norm(token_vec) * np.linalg.norm(final_token_vecs_array[i]))


[('شعاعي', nan),
 ('بطن', nan),
 ('تم', nan),
 ('صور', nan),
 ('[CLS]', -0.0),
 ('+ة', -0.0),
 ('ال+', 0.0),
 ('ل', nan),
 ('إجراء', nan),
 ('إذا', nan)]

# 5.3. Fine-tuning araBERTv02 for the project-specific data

In [31]:
!pip install virtualenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 kB 21.6 MB/s eta 0:00:00


In [32]:
!source my_env/bin/activate

/bin/bash: my_env/bin/activate: No such file or directory


In [33]:
!my_env\Scripts\activate

/bin/bash: my_envScriptsactivate: command not found


In [40]:
#!pip install fsspec==2021.8.1
!!pip install --upgrade fsspec

['Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/',
 'Requirement already satisfied: fsspec in /usr/local/lib/python3.10/dist-packages (2023.4.0)']

In [41]:
!!pip install datasets

['Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/',
 'Requirement already satisfied: datasets in /usr/local/lib/python3.10/dist-packages (2.12.0)',
 'Requirement already satisfied: fsspec[http]>=2021.11.1 in /usr/local/lib/python3.10/dist-packages (from datasets) (2023.4.0)',
 'Requirement already satisfied: pyarrow>=8.0.0 in /usr/local/lib/python3.10/dist-packages (from datasets) (9.0.0)',
 'Requirement already satisfied: packaging in /usr/local/lib/python3.10/dist-packages (from datasets) (23.1)',
 'Requirement already satisfied: aiohttp in /usr/local/lib/python3.10/dist-packages (from datasets) (3.8.4)',
 'Requirement already satisfied: dill<0.3.7,>=0.3.0 in /usr/local/lib/python3.10/dist-packages (from datasets) (0.3.6)',
 'Requirement already satisfied: pandas in /usr/local/lib/python3.10/dist-packages (from datasets) (1.5.3)',
 'Requirement already satisfied: numpy>=1.17 in /usr/local/lib/python3.10/dist-packages (from datasets) 

In [1]:
input_file_path = "/content/Talafeef--Seg.txt"
output_file_path = "/content/preprocessed_dataset.txt"

with open(input_file_path, "r", encoding="utf-8") as input_file:
    text = input_file.read()

sentences = text.split(". ")

with open(output_file_path, "w", encoding="utf-8") as output_file:
    for sentence in sentences:
        output_file.write(sentence.strip() + "\n")

In [3]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import load_dataset

def prepare_dataset(file_path, tokenizer):
    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)

    dataset = load_dataset("text", data_files={"train": file_path})
    tokenized_dataset = dataset.map(tokenize_function, batched=True, num_proc=4)
    return tokenized_dataset

tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv02")
file_path = "/content/preprocessed_dataset.txt"
tokenized_dataset = prepare_dataset(file_path, tokenizer)

model = AutoModelForMaskedLM.from_pretrained("aubmindlab/bert-base-arabertv02")

training_args = TrainingArguments(
    output_dir="./output",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    save_steps=10_000,
    save_total_limit=2,
    logging_steps=200,
    logging_dir="./logs",
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

  0%|          | 0/1 [00:00<?, ?it/s]

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
200,2.131900
400,1.837200


TrainOutput(global_step=525, training_loss=1.917574724469866, metrics={'train_runtime': 566.6275, 'train_samples_per_second': 7.423, 'train_steps_per_second': 0.927, 'total_flos': 1104838929678336.0, 'train_loss': 1.917574724469866, 'epoch': 2.99})

In [4]:
print("Number of examples in the tokenized dataset:", len(tokenized_dataset["train"]))

Number of examples in the tokenized dataset: 1402


In [6]:
!zip -r fine_tuned_arabertv02.zip fine_tuned_arabertv02
!pip install google-colab
from google.colab import files
files.download('fine_tuned_arabertv02.zip')

  adding: fine_tuned_arabertv02/ (stored 0%)
  adding: fine_tuned_arabertv02/training_args.bin (deflated 49%)
  adding: fine_tuned_arabertv02/tokenizer_config.json (deflated 42%)
  adding: fine_tuned_arabertv02/tokenizer.json (deflated 74%)
  adding: fine_tuned_arabertv02/generation_config.json (deflated 8%)
  adding: fine_tuned_arabertv02/vocab.txt (deflated 65%)
  adding: fine_tuned_arabertv02/config.json (deflated 47%)
  adding: fine_tuned_arabertv02/special_tokens_map.json (deflated 42%)
  adding: fine_tuned_arabertv02/pytorch_model.bin (deflated 7%)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.4 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
from transformers import pipeline

fill_mask = pipeline("fill-mask", model=model, tokenizer=tokenizer)

In [14]:
import torch
from transformers import pipeline
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
fill_mask = pipeline("fill-mask", model=model, tokenizer=tokenizer, device=0 if device == "cuda" else -1)

In [16]:
masked_input = "ومن الأشياء التي تعزز النظر، التفكير الجيد، فنمط  [MASK] هو الذي يسمح لنا بفهم الشروط الموضوعية والذاتية للسلوك الإنساني."
result = fill_mask(masked_input)
print("Result:", result)

Result: [{'score': 0.9823690056800842, 'token': 6537, 'token_str': 'التفكير', 'sequence': 'ومن الأشياء التي تعزز النظر ، التفكير الجيد ، فنمط التفكير هو الذي يسمح لنا بفهم الشروط الموضوعية والذاتية للسلوك الإنساني.'}, {'score': 0.004090497735887766, 'token': 16645, 'token_str': 'الفهم', 'sequence': 'ومن الأشياء التي تعزز النظر ، التفكير الجيد ، فنمط الفهم هو الذي يسمح لنا بفهم الشروط الموضوعية والذاتية للسلوك الإنساني.'}, {'score': 0.0034380166325718164, 'token': 2320, 'token_str': 'النظر', 'sequence': 'ومن الأشياء التي تعزز النظر ، التفكير الجيد ، فنمط النظر هو الذي يسمح لنا بفهم الشروط الموضوعية والذاتية للسلوك الإنساني.'}, {'score': 0.002521282061934471, 'token': 15514, 'token_str': 'تفكير', 'sequence': 'ومن الأشياء التي تعزز النظر ، التفكير الجيد ، فنمط تفكير هو الذي يسمح لنا بفهم الشروط الموضوعية والذاتية للسلوك الإنساني.'}, {'score': 0.0017259495798498392, 'token': 4136, 'token_str': 'الفكر', 'sequence': 'ومن الأشياء التي تعزز النظر ، التفكير الجيد ، فنمط الفكر هو الذي يسمح لنا ب

In [17]:
import os
import zipfile
from sklearn.model_selection import train_test_split
from datasets import Dataset


def prepare_dataset(examples, tokenizer):
    tokenized_examples = tokenizer(examples, padding="max_length", truncation=True)

    # Convert the tokenized examples to a list of dictionaries
    tokenized_examples_list = [dict(zip(tokenized_examples.keys(), col)) for col in zip(*tokenized_examples.values())]

    # Split the dataset into train, validation, and test sets
    train_size = 0.8
    val_size = 0.1

    train_list, temp_list = train_test_split(tokenized_examples_list, train_size=train_size, shuffle=True, random_state=42)
    val_list, test_list = train_test_split(temp_list, train_size=val_size/(1-train_size), shuffle=True, random_state=42)

    # Convert the lists of dictionaries back to Dataset objects
    train_dataset = Dataset.from_dict({key: [d[key] for d in train_list] for key in tokenized_examples.keys()})
    val_dataset = Dataset.from_dict({key: [d[key] for d in val_list] for key in tokenized_examples.keys()})
    test_dataset = Dataset.from_dict({key: [d[key] for d in test_list] for key in tokenized_examples.keys()})

    return train_dataset, val_dataset, test_dataset


file_path = "/content/preprocessed_dataset.txt"

# Read the text file and split it into a list of examples
with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()
examples = text.strip().split('\n')
examples = [example.strip() for example in examples if example.strip()]

print("Number of examples after filtering empty examples:", len(examples))

# Prepare the datasets
tokenized_train_dataset, tokenized_val_dataset, tokenized_test_dataset = prepare_dataset(examples, tokenizer)

trainer.train_dataset = tokenized_train_dataset
trainer.eval_dataset = tokenized_val_dataset

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print("Evaluation results:", eval_results)

# Test the model
masked_input = "ومن الأشياء التي تعزز النظر، التفكير الجيد، فنمط  [MASK] هو الذي يسمح لنا بفهم الشروط الموضوعية والذاتية للسلوك الإنساني."
result = fill_mask(masked_input)
print("Result:", result)

Number of examples after filtering empty examples: 1396


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


Evaluation results: {'eval_loss': 1.4778741598129272, 'eval_runtime': 6.491, 'eval_samples_per_second': 21.568, 'eval_steps_per_second': 2.773, 'epoch': 3.0}
Result: [{'score': 0.9871422052383423, 'token': 6537, 'token_str': 'التفكير', 'sequence': 'ومن الأشياء التي تعزز النظر ، التفكير الجيد ، فنمط التفكير هو الذي يسمح لنا بفهم الشروط الموضوعية والذاتية للسلوك الإنساني.'}, {'score': 0.00380714051425457, 'token': 15514, 'token_str': 'تفكير', 'sequence': 'ومن الأشياء التي تعزز النظر ، التفكير الجيد ، فنمط تفكير هو الذي يسمح لنا بفهم الشروط الموضوعية والذاتية للسلوك الإنساني.'}, {'score': 0.0029500385280698538, 'token': 2320, 'token_str': 'النظر', 'sequence': 'ومن الأشياء التي تعزز النظر ، التفكير الجيد ، فنمط النظر هو الذي يسمح لنا بفهم الشروط الموضوعية والذاتية للسلوك الإنساني.'}, {'score': 0.001025404199026525, 'token': 16645, 'token_str': 'الفهم', 'sequence': 'ومن الأشياء التي تعزز النظر ، التفكير الجيد ، فنمط الفهم هو الذي يسمح لنا بفهم الشروط الموضوعية والذاتية للسلوك الإنساني.'}, {